<a href="https://colab.research.google.com/github/mbhargava4/CS-539-Final-Project-Equation-Solver/blob/main/Convert_inkml_to_png.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage.transform import resize
import xml.etree.ElementTree as ET
import os
from tqdm import tqdm
import cv2
!pip install split-folders
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [30]:
path = '/content/drive/MyDrive/ECE_class/CS 539 Group /data_inkml'
def seed_everything(seed):

    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)


SEED = 999
seed_everything(SEED)

In [31]:
def get_traces_data(inkml_file_abs_path):


    traces_data = []

    tree = ET.parse(inkml_file_abs_path)
    root = tree.getroot()
    doc_namespace = "{http://www.w3.org/2003/InkML}"
    traces_all = [{'id': trace_tag.get('id'),
    					'coords': [[round(float(axis_coord)) if float(axis_coord).is_integer() else round(float(axis_coord) * 10000) \
    									for axis_coord in coord[1:].split(' ')] if coord.startswith(' ') \
    								else [round(float(axis_coord)) if float(axis_coord).is_integer() else round(float(axis_coord) * 10000) \
    									for axis_coord in coord.split(' ')] \
    							for coord in (trace_tag.text).replace('\n', '').split(',')]} \
    							for trace_tag in root.findall(doc_namespace + 'trace')]

    traces_all.sort(key=lambda trace_dict: int(trace_dict['id']))
    traceGroupWrapper = root.find(doc_namespace + 'traceGroup')

    if traceGroupWrapper is not None:
        for traceGroup in traceGroupWrapper.findall(doc_namespace + 'traceGroup'):

            label = traceGroup.find(doc_namespace + 'annotation').text

            traces_curr = []
            for traceView in traceGroup.findall(doc_namespace + 'traceView'):

                traceDataRef = int(traceView.get('traceDataRef'))

                single_trace = traces_all[traceDataRef]['coords']
                traces_curr.append(single_trace)

            traces_data.append({'label': label, 'trace_group': traces_curr})

    else:
        [traces_data.append({'trace_group': [trace['coords']]}) for trace in traces_all]

    return traces_data

In [32]:
def inkml2img(input_path, output_path):
    traces = get_traces_data(input_path)
    path_parts = input_path.split('/')
    filename = path_parts[-1].split('.')[0]

    for i, elem in enumerate(traces):
        plt.gca().invert_yaxis()
        plt.gca().set_aspect('equal', adjustable='box')
        plt.axis('off')
        plt.plot([0], [0])
        ls = elem['trace_group']
        for subls in ls:
            data = np.array(subls)
            x, y = zip(*data)
            plt.plot(x, y, linewidth=2, c='black')

        capital_list = ['A', 'B', 'C', 'F', 'X', 'Y']
        label = 'capital_' + elem['label'] if elem['label'] in capital_list else elem['label']
        ind_output_path = os.path.join(output_path, label)

        os.makedirs(ind_output_path, exist_ok=True)

        filepath = os.path.join(ind_output_path, f"{filename}_{i}.png")
        if not os.path.exists(filepath):
            plt.savefig(filepath, bbox_inches='tight', pad_inches=0, dpi=100)
        plt.clf()  # Clear current figure

In [33]:
import os
image_data_folder = '/kaggle/Image_data'
final_train_folder = '/kaggle/Image_data/finaltrain'

if not os.path.exists(image_data_folder):
    os.mkdir(image_data_folder)
    print("Folder 'Image_data' created successfully at:", image_data_folder)
else:
    print("Folder 'Image_data' already exists at:", image_data_folder)

if not os.path.exists(final_train_folder):
    os.mkdir(final_train_folder)
    print("Folder 'finaltrain' created successfully at:", final_train_folder)
else:
    print("Folder 'finaltrain' already exists at:", final_train_folder)


Folder 'Image_data' already exists at: /kaggle/Image_data
Folder 'finaltrain' already exists at: /kaggle/Image_data/finaltrain


In [36]:
files = os.listdir(os.getcwd() + '/drive/MyDrive/ECE_class/CS 539 Group /data_inkml/CROHME_training_2011')
for file in tqdm(files):
    inkml2img(path + '/CROHME_training_2011/' + file, '/content/drive/MyDrive/ECE_class/CS 539 Group /processed_data')

/content/drive/MyDrive/ECE_class/CS 539 Group /data_inkml
['TrainData2_1_sub_9.inkml', 'TrainData2_18_sub_49.inkml', 'TrainData2_7_sub_98.inkml', 'TrainData2_7_sub_88.inkml', 'formulaire008-equation051.inkml', 'TrainData2_20_sub_73.inkml', 'TrainData2_14_sub_39.inkml', 'TrainData2_26_sub_73.inkml', 'TrainData1_7_sub_21.inkml', 'TrainData2_11_sub_17.inkml', 'TrainData2_12_sub_9.inkml', 'TrainData2_14_sub_33.inkml', 'TrainData2_21_sub_98.inkml', 'formulaire002-equation006.inkml', 'TrainData2_4_sub_98.inkml', 'TrainData2_10_sub_95.inkml', 'TrainData2_7_sub_9.inkml', 'TrainData2_23_sub_20.inkml', 'TrainData2_22_sub_9.inkml', 'formulaire004-equation048.inkml', 'TrainData2_21_sub_49.inkml', 'TrainData2_18_sub_46.inkml', 'TrainData2_4_sub_39.inkml', 'formulaire006-equation003.inkml', 'TrainData2_2_sub_33.inkml', 'TrainData2_22_sub_17.inkml', 'formulaire001-equation036.inkml', 'TrainData2_10_sub_73.inkml', 'TrainData2_13_sub_29.inkml', 'formulaire006-equation051.inkml', 'formulaire007-equation

100%|██████████| 921/921 [16:25<00:00,  1.07s/it]


<Figure size 640x480 with 0 Axes>